In [ ]:
# Checking the features used for SVD and VOICED datasets
import pandas as pd
svd = pd.read_csv("data/flattened_features.csv")
voiced = pd.read_csv("data/voiced_features_8000_fft.csv")

svd_cols = set(svd.columns.to_list())
voiced_cols = set(voiced.columns.to_list())
svd_cols - voiced_cols

In [5]:
# Comparing new and old results
with open("latex_tables.tex", "r") as f1:
    content1 = f1.read()

with open("latex_tables_old.tex", "r") as f2:
    content2 = f2.read()

content1 == content2

False

In [12]:
# Statistical testing of differences between max / min differences for each combination of scaling transformation and dataset
import pandas as pd
from scipy.stats import wilcoxon, normaltest
from pathlib import Path
datasets = ["svd", "voiced"]

filename_list = list([file.name for file in Path("svd_results").glob("*.json")])

results_table = pd.DataFrame(data=[], columns=["dataset", "transformer", "model", "p_value"])
for dataset in datasets:
    for name in filename_list:
        model = name.split(".")[0].split("_1000_")[1]
        transformer = name.split("_")[1]
        path_random = Path(f"{dataset}_results", name)
        path_stratified = Path(f"{dataset}_results_stratified", name)
        data_random = pd.read_json(path_random).transpose()
        data_stratified = pd.read_json(path_stratified).transpose()
        data_random = pd.json_normalize(data_random["bcc"])
        data_stratified = pd.json_normalize(data_stratified["bcc"])
        data_random["diff"] = data_random["leakage"] - data_random["correct"]
        data_stratified["diff"] = data_stratified["leakage"] - data_stratified["correct"]

        for split, data in zip(["random", "stratified"], [data_random, data_stratified]):
            _, p_value = normaltest(data["diff"])
            print(f"{dataset:<6} {split:<10} {transformer:<20} {model:<20} {p_value:.3f}")
        # _, p_value = wilcoxon(data_random["diff"], data_stratified["diff"])
        # results_table.loc[results_table.shape[0], :] = [dataset, transformer, model, p_value]

# results_table.to_csv("test_stratification_effect.csv", index=False)



svd    random     MaxAbsScaler         adaboost             nan
svd    stratified MaxAbsScaler         adaboost             nan
svd    random     MaxAbsScaler         dt                   nan
svd    stratified MaxAbsScaler         dt                   nan
svd    random     MaxAbsScaler         gaussianNB           nan
svd    stratified MaxAbsScaler         gaussianNB           nan
svd    random     MaxAbsScaler         gaussian_process     0.000
svd    stratified MaxAbsScaler         gaussian_process     0.000
svd    random     MaxAbsScaler         knn                  0.049
svd    stratified MaxAbsScaler         knn                  0.046
svd    random     MaxAbsScaler         lda                  nan
svd    stratified MaxAbsScaler         lda                  nan
svd    random     MaxAbsScaler         mlp                  0.198
svd    stratified MaxAbsScaler         mlp                  0.633
svd    random     MaxAbsScaler         qda                  0.000
svd    stratified MaxAbsSc